# 📊 Data Cleaning Pipeline

**Step 4: Clean the Data**

This notebook covers:
- 4.1 Load and Inspect
- 4.2 Remove Cancellations and Bad Values
- 4.3 Standardize Descriptions
- 4.4 Add Derived Columns
- 4.5 Save Clean Files

---

In [34]:
from pathlib import Path
def project_root(start: Path = None) -> Path:
    here = start or Path.cwd()
    for p in [here, *here.parents]:
        if (p / "data_raw").exists() or (p / "outputs").exists() or (p / "data_clean").exists():
            return p
    return here
ROOT = project_root()
DATA_CLEAN = ROOT / "data_clean"
OUTPUTS = ROOT / "outputs"
DATA_CLEAN.mkdir(parents=True, exist_ok=True)
OUTPUTS.mkdir(parents=True, exist_ok=True)
print("ROOT:", ROOT)
print("DATA_CLEAN:", DATA_CLEAN)
print("OUTPUTS:", OUTPUTS)


ROOT: /Users/alihasan/retail-pricing-mba
DATA_CLEAN: /Users/alihasan/retail-pricing-mba/data_clean
OUTPUTS: /Users/alihasan/retail-pricing-mba/outputs


In [35]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
# Set plotting style - use a compatible style
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')
        print("⚠️  Using default matplotlib style")

# Set seaborn style safely
try:
    sns.set_palette("husl")
    sns.set_style("whitegrid")
except:
    print("⚠️  Using default seaborn settings")

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [36]:
# ---------------------------
# 📂 Set up project root paths
# ---------------------------
from pathlib import Path

def project_root():
    """
    Walks up folders until it finds your project root
    (the one with data_raw/ and data_clean/).
    """
    here = Path.cwd()
    for p in [here, *here.parents]:
        if (p / "data_raw").exists():
            return p
    return here

ROOT = project_root()
DATA_CLEAN = ROOT / "data_clean"
DATA_CLEAN.mkdir(parents=True, exist_ok=True)  # create if missing

print(f"📂 Project root set to: {ROOT}")
print(f"📂 Clean data folder: {DATA_CLEAN}")


📂 Project root set to: /Users/alihasan/retail-pricing-mba
📂 Clean data folder: /Users/alihasan/retail-pricing-mba/data_clean


## 4.1 Load and Inspect Data

In [37]:
# Import our inspection function from src module
import sys
import os

# Fix the path issue - we need to go up one level from notebooks/
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    # We're in notebooks folder, go up one level
    os.chdir('..')
    print(f"📁 Changed working directory to: {os.getcwd()}")

from src.data_inspection import load_and_inspect_data

# 4.1 Load and inspect (using our function)
df_raw = load_and_inspect_data()

if df_raw is not None:
    print(f"✅ Raw data loaded: {len(df_raw):,} rows × {len(df_raw.columns)} columns")
    print(f"📅 Date range: {df_raw['InvoiceDate'].min()} to {df_raw['InvoiceDate'].max()}")
else:
    print("❌ Failed to load data. Please check the file path.")
    exit()

Loading online retail data...
✅ Data loaded successfully!
📊 Shape: 525461 rows × 8 columns

🔍 First 5 rows:
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  

📋 Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total

## 4.2 Remove Cancellations and Bad Values

In [38]:
# Create a copy for cleaning
df_clean = df_raw.copy()
print(f"📊 Starting with {len(df_clean):,} rows")
print(f"💾 Memory usage: {df_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

📊 Starting with 525,461 rows
💾 Memory usage: 131.6 MB


In [39]:
# Remove cancellations (invoices starting with 'C')
cancellations = df_clean['Invoice'].str.startswith('C', na=False).sum()
df_clean = df_clean[~df_clean['Invoice'].str.startswith('C', na=False)]
print(f"🗑️  Removed {cancellations:,} cancellation rows")
print(f"📊 Rows after removing cancellations: {len(df_clean):,}")

🗑️  Removed 10,206 cancellation rows
📊 Rows after removing cancellations: 515,255


In [40]:
# Remove rows with negative or zero quantities and prices
bad_quantities = (df_clean['Quantity'] <= 0).sum()
bad_prices = (df_clean['Price'] <= 0).sum()

df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['Price'] > 0)]
print(f"🗑️  Removed {bad_quantities:,} rows with bad quantities (≤0)")
print(f"🗑️  Removed {bad_prices:,} rows with bad prices (≤0)")
print(f"📊 Rows after removing bad values: {len(df_clean):,}")

🗑️  Removed 2,121 rows with bad quantities (≤0)
🗑️  Removed 3,690 rows with bad prices (≤0)
📊 Rows after removing bad values: 511,565


In [41]:
# Remove rows with missing Customer ID or Description
missing_customer = df_clean['Customer ID'].isnull().sum()
missing_description = df_clean['Description'].isnull().sum()

df_clean = df_clean.dropna(subset=['Customer ID', 'Description'])
print(f"🗑️  Removed {missing_customer:,} rows with missing Customer ID")
print(f"🗑️  Removed {missing_description:,} rows with missing Description")
print(f"📊 Rows after removing missing data: {len(df_clean):,}")

🗑️  Removed 103,901 rows with missing Customer ID
🗑️  Removed 0 rows with missing Description
📊 Rows after removing missing data: 407,664


## 4.3 Standardize Descriptions

In [42]:
# Standardize product descriptions
print("🔧 Standardizing product descriptions...")
df_clean['Description'] = df_clean['Description'].str.strip().str.title()
print("✅ Product descriptions standardized")

🔧 Standardizing product descriptions...
✅ Product descriptions standardized


## 4.4 Add Derived Columns

In [43]:
# Convert InvoiceDate to datetime if not already
print("📅 Converting InvoiceDate to datetime...")
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
print("✅ InvoiceDate converted to datetime")

📅 Converting InvoiceDate to datetime...
✅ InvoiceDate converted to datetime


In [44]:
# Add TotalPrice column
print("💰 Adding TotalPrice column...")
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['Price']
print("✅ TotalPrice column added")

💰 Adding TotalPrice column...
✅ TotalPrice column added


In [45]:
# Add InvoiceYearMonth for monthly trends
print("📅 Adding InvoiceYearMonth column...")
df_clean['InvoiceYearMonth'] = df_clean['InvoiceDate'].dt.to_period('M').astype(str)
print("✅ InvoiceYearMonth column added")

📅 Adding InvoiceYearMonth column...
✅ InvoiceYearMonth column added


In [46]:
import os
print(os.getcwd())


/Users/alihasan/retail-pricing-mba


## 4.5 Save Clean Files

In [47]:
# Save cleaned transactions
df_clean.to_csv(DATA_CLEAN / "transactions.csv", index=False)
print(f"✅ Cleaned transactions saved to: {DATA_CLEAN / 'transactions.csv'}")

✅ Cleaned transactions saved to: /Users/alihasan/retail-pricing-mba/data_clean/transactions.csv


In [48]:
# ---------------------------
# Create and save dimension tables (robust)
# ---------------------------

from pathlib import Path

# Make sure DATA_CLEAN exists (you already defined ROOT/DATA_CLEAN at the top)
DATA_CLEAN.mkdir(parents=True, exist_ok=True)

# Handle possible column name variants for Customer ID
cust_col = None
for candidate in ["Customer ID", "Customer_ID", "CustomerID", "Customer Id"]:
    if candidate in df_clean.columns:
        cust_col = candidate
        break
if cust_col is None:
    raise KeyError("Could not find a Customer ID column in df_clean. "
                   "Tried: 'Customer ID', 'Customer_ID', 'CustomerID', 'Customer Id'.")

# Build dimension frames
products_df = df_clean[["StockCode", "Description"]].drop_duplicates()
customers_df = df_clean[[cust_col, "Country"]].drop_duplicates()
customers_df = customers_df.rename(columns={cust_col: "Customer_ID"})  # standardize name

# Save using the resolved project path
products_df.to_csv(DATA_CLEAN / "products.csv", index=False)
customers_df.to_csv(DATA_CLEAN / "customers.csv", index=False)

print(f"✅ Products: {len(products_df):,}, Customers: {len(customers_df):,}")
print(f"📄 Saved: {DATA_CLEAN / 'products.csv'}")
print(f"📄 Saved: {DATA_CLEAN / 'customers.csv'}")
print("🎉 Data cleaning complete!")


✅ Products: 4,445, Customers: 4,317
📄 Saved: /Users/alihasan/retail-pricing-mba/data_clean/products.csv
📄 Saved: /Users/alihasan/retail-pricing-mba/data_clean/customers.csv
🎉 Data cleaning complete!
